# Jupyter Notebook implementation

First read the [README](README.md) file if you're new.

This is an example of using the code from Jupyter Notebook.

## Code Tree

- `Pointnet_Pointnet2_pytorch/`
  - `data/*`: Default location for data.
  - `data_utils/*`: Data Loader.
  - `models/*`: Model file, `torch.nn.Module` classes. See below for more details.
  - `log/*`: log and output of trained model.
  - `visualize/*`: Visualization code. This is independent of the main code.
  - `test_*.py`: Predeiction codes.
  - `train_*.py`: Training codes.

## Data directory:

This could be named anything and placed anywhere, as long as its path is passed to the argument.

- `data/`: Data directory, create this manually.
  - `modelnet40_normal_resampled/*`: ModelNet40 dataset.
  - `shapenetcore_partanno_segmentation_benchmark_v0_normal/*`: ShapeNet dataset.
  - `Stanford3dDataset_v1.2_Aligned_Version/*`: S3DIS dataset.
  - (`stanford_indoor3d/*`): S3DIS dataset converted by `data_utils/collect_indoor3d_data.py`.

For details, see below sections for each model.

## Models

When running `main()`, the argument `--model` will specify the model to be used.
It will search the directory `models/` for a file name that matches the argument.
Create a new file in the `models/` directory to add a new model.

### Aliases

Core Model

- `pointnet_*.py`: PointNet model
- `pointnet2_*.py`: PointNet++ model

Task Type

- `*_cls*.py`: Classification model
- `*_sem_seg*.py`: Segmentation model
- `*_part_seg*.py`: Part segmentation model

Grouping Method

- `*_msg.py`: Multi-scale grouping model
- `*_ssg.py`: Single-scale grouping model

### Model Directory Tree

- `Pointnet_Pointnet2_pytorch/models/`
  - `pointnet_cls.py`: PointNet classification model
  - `pointnet_part_seg.py`: PointNet part segmentation model
  - `pointnet_sem_seg.py`: PointNet semantic segmentation model
  - `pointnet_utils.py`: PointNet util functions
  - `pointnet2_cls_msg.py`: PointNet++ classification model with multi-scale grouping
  - `pointnet2_cls_ssg.py`: PointNet++ classification model with single-scale grouping
  - `pointnet2_part_seg_msg.py`: PointNet++ part segmentation model with multi-scale grouping
  - `pointnet2_part_seg_ssg.py`: PointNet++ part segmentation model with single-scale grouping
  - `pointnet2_sem_seg_msg.py`: PointNet++ semantic segmentation model with multi-scale grouping
  - `pointnet2_sem_seg_ssg.py`: PointNet++ semantic segmentation model with single-scale grouping
  - `pointnet2_utils.py`: PointNet++ util functions

## Imports

In [4]:
from pprint import pprint
from pathlib import Path

import train_classification
import test_classification

import train_partseg
import test_partseg

import train_semseg
import test_semseg
from data_utils.collect_indoor3d_data import collect_indoor3d_data

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


## **Optional**: External Directory

If you want to put the log and data directories in a different location, you can use the following code.

In [3]:
root_log_dir = Path('..', 'log').resolve()
DATA_DIR = str(Path('..', 'data').resolve())
class_data_dir = str(Path(DATA_DIR, 'modelnet40_normal_resampled').resolve())
partseg_data_dir = str(Path(DATA_DIR, 'shapenetcore_partanno_segmentation_benchmark_v0_normal').resolve())
semseg_raw_data_dir = str(Path(DATA_DIR, 's3dis', 'Stanford3dDataset_v1.2_Aligned_Version').resolve())
semseg_data_dir = str(Path(DATA_DIR, 'stanford_indoor3d').resolve())

root_log_dir, DATA_DIR, class_data_dir, partseg_data_dir, semseg_raw_data_dir, semseg_data_dir

(PosixPath('/home/gokitune/Documents/pointnet-cleanup/log'),
 '/home/gokitune/Documents/pointnet-cleanup/data',
 '/home/gokitune/Documents/pointnet-cleanup/data/modelnet40_normal_resampled',
 '/home/gokitune/Documents/pointnet-cleanup/data/shapenetcore_partanno_segmentation_benchmark_v0_normal',
 '/home/gokitune/Documents/pointnet-cleanup/data/s3dis/Stanford3dDataset_v1.2_Aligned_Version',
 '/home/gokitune/Documents/pointnet-cleanup/data/stanford_indoor3d')

---

## Classification (ModelNet)


### Data

- ModelNet40 dataset in text format
- Download `modelnet40_normal_resampled` from:
  - [Kaggle](https://www.kaggle.com/datasets/chenxaoyu/modelnet-normal-resampled)  
   [Pointcept Huggingface](https://huggingface.co/datasets/Pointcept/modelnet40_normal_resampled-compressed)

Default `--data_dir` is `'data/modelnet40_normal_resampled'`.

- `data/`
  - `modelnet40_normal_resampled/`
    - `train/`
      - `airplane/`
        - `airplane_0001.txt`
        - `airplane_0002.txt`
        - ...
    - ...
    - `file_list.txt`
    - `modelnet10_shape_names.txt`
    - `modelnet10_train.txt`
    - `modelnet10_test.txt`
    - `modelnet40_shape_names.txt`
    - `modelnet40_train.txt`
    - `modelnet40_test.txt`

### **OUTPUT**

- TRAINING: `<log_root>/classification/<args.log_dir or TIME>/checkpoints/best_model.pth`
- PREDICTION: `None (only prints accuracy)`


### Training

`train_classification.py` is used to train the model.

Check all the arguments:

In [3]:
!python train_classification.py -h

usage: training [-h] [--use_cpu] [--gpu GPU] [--batch_size BATCH_SIZE]
                [--model MODEL] [--num_category {10,40}] [--epoch EPOCH]
                [--learning_rate LEARNING_RATE] [--num_point NUM_POINT]
                [--optimizer OPTIMIZER] [--log_root LOG_ROOT]
                [--log_dir LOG_DIR] [--decay_rate DECAY_RATE] [--use_normals]
                [--process_data] [--use_uniform_sample] [--data_dir DATA_DIR]

options:
  -h, --help            show this help message and exit
  --use_cpu             use cpu mode
  --gpu GPU             specify gpu device
  --batch_size BATCH_SIZE
                        batch size in training
  --model MODEL         model name [default: pointnet_cls]
  --num_category {10,40}
                        training on ModelNet10/40
  --epoch EPOCH         number of epoch in training
  --learning_rate LEARNING_RATE
                        learning rate in training
  --num_point NUM_POINT
                        Point Number
  --optimizer OPTI

The following is the same as running:

```bash
python3 train_classification.py \
     --model pointnet2_cls_ssg \
     --log_dir pointnet2_cls_ssg \
     # --log_root ../log \
     # --data_dir ../data/modelnet40_normal_resampled
```

In [4]:
# user defined arguments for command line as a dictionary
args = {
    'model'   : 'pointnet2_cls_ssg',
    'log_dir' : 'pointnet2_cls_ssg',
    'log_root': root_log_dir,
    'data_dir': class_data_dir,
}
classification_train_args = train_classification.CommandLineArgs(**args)
train_classification.main(classification_train_args)

PARAMETER ...
CommandLineArgs(model='pointnet2_cls_ssg', log_dir='pointnet2_cls_ssg', log_root=PosixPath('/home/gokitune/Documents/pointnet-cleanup/log'), data_dir='/home/gokitune/Documents/pointnet-cleanup/data/modelnet40_normal_resampled', use_cpu=False, gpu='0', batch_size=25, num_category=40, epoch=200, learning_rate=0.001, num_point=1024, optimizer='adam', decay_rate=0.0001, use_normals=False, process_data=False, use_uniform_sample=False)
Load dataset ...
The size of train data is 9843
The size of test data is 2468
No existing model, starting training from scratch...
Epoch 1 (1/200):


100%|██████████| 393/393 [01:24<00:00,  4.67it/s]

Train Instance Accuracy: 0.495471



100%|██████████| 99/99 [00:17<00:00,  5.52it/s]


Test Instance Accuracy: 0.618721, Class Accuracy: 0.538219
Best Instance Accuracy: 0.618721, Class Accuracy: 0.538219
Saving at /home/gokitune/Documents/pointnet-cleanup/log/classification/pointnet2_cls_ssg/checkpoints/best_model.pth
Epoch 2 (2/200):


...

Test Instance Accuracy: 0.914927, Class Accuracy: 0.885296
Best Instance Accuracy: 0.923973, Class Accuracy: 0.896426
Epoch 200 (200/200):



100%|██████████| 393/393 [01:23<00:00,  4.69it/s]

Train Instance Accuracy: 0.968550



100%|██████████| 99/99 [00:18<00:00,  5.29it/s]

Test Instance Accuracy: 0.913468, Class Accuracy: 0.886938
Best Instance Accuracy: 0.923973, Class Accuracy: 0.896426


### Testing

`test_classification.py` is used to test the model.

Check all the arguments:

In [3]:
!python test_classification.py -h

usage: Testing [-h] [--use_cpu] [--gpu GPU] [--batch_size BATCH_SIZE]
               [--num_category {10,40}] [--num_point NUM_POINT]
               [--log_root LOG_ROOT] --log_dir LOG_DIR [--use_normals]
               [--use_uniform_sample] [--num_votes NUM_VOTES]
               [--data_dir DATA_DIR]

options:
  -h, --help            show this help message and exit
  --use_cpu             use cpu mode
  --gpu GPU             specify gpu device
  --batch_size BATCH_SIZE
                        batch size in training
  --num_category {10,40}
                        training on ModelNet10/40
  --num_point NUM_POINT
                        Point Number
  --log_root LOG_ROOT   Log directory root [default: log]
  --log_dir LOG_DIR     Experiment root within log directory
  --use_normals         use normals
  --use_uniform_sample  use uniform sampiling
  --num_votes NUM_VOTES
                        Aggregate classification scores with voting
  --data_dir DATA_DIR   data directory [default:

The following is the same as running:

```bash
python3 test_classification.py \
     --log_dir pointnet2_cls_ssg \
     # --log_root ../log \
     # --data_dir ../data/modelnet40_normal_resampled
```

In [4]:
args = {
    'log_dir': 'pointnet2_cls_ssg',
    'log_root': root_log_dir,
    'data_dir': class_data_dir,
}
classification_test_args = test_classification.CommandLineArgs(**args)
test_classification.main(classification_test_args)

PARAMETER ...
CommandLineArgs(log_dir='pointnet2_cls_ssg', log_root=PosixPath('/home/gokitune/Documents/pointnet-cleanup/log'), data_dir='/home/gokitune/Documents/pointnet-cleanup/data/modelnet40_normal_resampled', use_cpu=False, gpu='0', batch_size=24, num_category=40, num_point=1024, use_normals=False, use_uniform_sample=False, num_votes=3)
Load dataset ...
The size of test data is 2468


100%|██████████| 103/103 [00:57<00:00,  1.78it/s]

Test Instance Accuracy: 0.924434, Class Accuracy: 0.890748


---

## Part Segmentation (ShapeNet)

## Data

- ShapeNet dataset in text format
- Download `shapenetcore_partanno_segmentation_benchmark_v0_normal` from:
  - [Kaggle](https://www.kaggle.com/datasets/mitkir/shapenet?resource=download)


Default `--data_dir` is `'data/shapenetcore_partanno_segmentation_benchmark_v0_normal'`.

- `data/`
  - `shapenetcore_partanno_segmentation_benchmark_v0_normal/`
    - `02691156/`
      - `1a04e3eab45ca15dd86060f189eb133.txt`
      - `1a04e3eab45ca15dd86060f189eb133.npz`
      - ...
    - ...
    - `train_test_split/`
      - `shuffled_train_filelist.json`
      - `shuffled_val_filelist.json`
      - `shuffled_test_filelist.json`
    - `utils/`
      - `potential_field.py`
      - `som.py`
    - `synsetoffset2category.txt`

### Data structure

From `data/shapenetcore_partanno_segmentation_benchmark_v0_normal/synsetoffset2category.txt`, the folders correspond to the following categories:

- `Airplane`: 02691156
- `Bag`: 02773838
- `Cap`: 02954340
- `Car`: 02958343
- `Chair`: 03001627
- `Earphone`: 03261776
- `Guitar`: 03467517
- `Knife`: 03624134
- `Lamp`: 03636649
- `Laptop`: 03642806
- `Motorbike`: 03790512
- `Mug`: 03797390
- `Pistol`: 03948459
- `Rocket`: 04099429
- `Skateboard`: 04225987
- `Table`: 04379243

For each .txt file within the folder above, 

 - `[i, :]` is the i th point.
 - `[:, 0:3]` is xyz.
 - `[:, 3:6]` is normalized xyz.
 - `[:, 6]` is the segmentation label.

i.e., each row is a point, and the columns are `[x, y, z, nx, ny, nz, label]`.

`--normal` flag will use all x-y-z-nx-ny-nz + label as input. Otherwise, only x-y-z + label will be used.

#### **OUTPUT**

- TRAINING: `<log_root>/part_seg/<args.log_dir or TIME>/checkpoints/best_model.pth`


### Define Segmentation Classes

In [3]:
# shapenet part segmentation
seg_classes = {
    'Earphone'  : [16, 17, 18],
    'Motorbike' : [30, 31, 32, 33, 34, 35],
    'Rocket'    : [41, 42, 43],
    'Car'       : [8, 9, 10, 11],
    'Laptop'    : [28, 29],
    'Cap'       : [6, 7],
    'Skateboard': [44, 45, 46],
    'Mug'       : [36, 37],
    'Guitar'    : [19, 20, 21],
    'Bag'       : [4, 5],
    'Lamp'      : [24, 25, 26, 27],
    'Table'     : [47, 48, 49],
    'Airplane'  : [0, 1, 2, 3],
    'Pistol'    : [38, 39, 40],
    'Chair'     : [12, 13, 14, 15],
    'Knife'     : [22, 23]
}

seg_ids = [seg_id for seg_val_sublist in seg_classes.values() for seg_id in seg_val_sublist]
len(seg_classes), len(seg_ids)

(16, 50)

### Training

`train_partseg.py` is used to train the model.

Check all the arguments:

In [6]:
!python3 train_partseg.py -h

usage: Model [-h] [--model MODEL] [--batch_size BATCH_SIZE] [--epoch EPOCH]
             [--learning_rate LEARNING_RATE] [--gpu GPU]
             [--optimizer OPTIMIZER] [--log_root LOG_ROOT] [--log_dir LOG_DIR]
             [--decay_rate DECAY_RATE] [--npoint NPOINT] [--normal]
             [--step_size STEP_SIZE] [--lr_decay LR_DECAY]
             [--data_dir DATA_DIR]

options:
  -h, --help            show this help message and exit
  --model MODEL         model name [default: pointnet_part_seg]
  --batch_size BATCH_SIZE
                        batch Size during training [default: 16]
  --epoch EPOCH         epoch to run [default: 251]
  --learning_rate LEARNING_RATE
                        initial learning rate [default: 0.001]
  --gpu GPU             specify GPU devices [default: 0]
  --optimizer OPTIMIZER
                        Adam or SGD [default: Adam]
  --log_root LOG_ROOT   Log root directory [default: log]
  --log_dir LOG_DIR     log path wihin log root directory
  --decay

The following is the same as running:

```bash
python3 train_partseg.py \
     --model pointnet2_part_seg_msg \
     --normal \
     --log_dir pointnet2_part_seg_msg \
     # --log_root ../log \
     # --data_dir ../data/shapenetcore_partanno_segmentation_benchmark_v0_normal
```

In [4]:
args = {
    'model'   : 'pointnet2_part_seg_msg',
    'normal'  : True, # in source: action='store_true'
    'log_dir' : 'pointnet2_part_seg_msg',
    'log_root': root_log_dir,
    'data_dir': partseg_data_dir,
}
partseg_train_args = train_partseg.CommandLineArgs(**args)
train_partseg.main(partseg_train_args, seg_classes)

PARAMETER ...
CommandLineArgs(model='pointnet2_part_seg_msg', normal=True, log_dir='pointnet2_part_seg_msg', log_root=PosixPath('/home/gokitune/Documents/pointnet-cleanup/log'), data_dir='/home/gokitune/Documents/pointnet-cleanup/data/shapenetcore_partanno_segmentation_benchmark_v0_normal', batch_size=16, epoch=251, learning_rate=0.001, gpu='0', optimizer='Adam', decay_rate=0.0001, npoint=2048, step_size=20, lr_decay=0.5)
The number of training data is: 13998
The number of test data is: 2874
No existing model, starting training from scratch...
Epoch 1 (1/251):
Learning rate:0.001000
BN momentum updated to: 0.100000


100%|██████████| 874/874 [05:25<00:00,  2.68it/s]

Train accuracy is: 0.85810



100%|██████████| 180/180 [00:46<00:00,  3.91it/s]

eval mIoU of Airplane       0.760997
eval mIoU of Bag            0.453149
eval mIoU of Cap            0.472498
eval mIoU of Car            0.582741
eval mIoU of Chair          0.866620
eval mIoU of Earphone       0.535288
eval mIoU of Guitar         0.877399
eval mIoU of Knife          0.828719
eval mIoU of Lamp           0.756419
eval mIoU of Laptop         0.935098
eval mIoU of Motorbike      0.175747
eval mIoU of Mug            0.731121
eval mIoU of Pistol         0.511420
eval mIoU of Rocket         0.245450
eval mIoU of Skateboard     0.481472
eval mIoU of Table          0.785021
Epoch 1 test Accuracy: 0.910146  Class avg mIOU: 0.624947   Inctance avg mIOU: 0.773584
Saving at /home/gokitune/Documents/pointnet-cleanup/log/part_seg/pointnet2_part_seg_msg/checkpoints/best_model.pth
Saving model....
Best accuracy is: 0.91015
Best class avg mIOU is: 0.62495
Best inctance avg mIOU is: 0.77358


{'accuracy': np.float64(0.9101457164448504),
 'class_avg_accuracy': np.float64(0.6349527879241524),
 'class_avg_iou': np.float64(0.6249473838873888),
 'inctance_avg_iou': np.float64(0.77358353431179)}

### Testing

`test_partseg.py` is used to test the model.

Check all the arguments:

In [14]:
!python3 test_partseg.py -h

usage: PointNet [-h] [--batch_size BATCH_SIZE] [--gpu GPU]
                [--num_point NUM_POINT] [--log_root LOG_ROOT] --log_dir
                LOG_DIR [--normal] [--num_votes NUM_VOTES]
                [--data_dir DATA_DIR]

options:
  -h, --help            show this help message and exit
  --batch_size BATCH_SIZE
                        batch size in testing [default: 24]
  --gpu GPU             specify gpu device [default: 0]
  --num_point NUM_POINT
                        point Number [default: 2048]
  --log_root LOG_ROOT   Log directory root [default: log]
  --log_dir LOG_DIR     experiment root within log directory
  --normal              use normals
  --num_votes NUM_VOTES
                        aggregate segmentation scores with voting [default: 3]
  --data_dir DATA_DIR   data directory [default: data/shapenetcore_partanno_se
                        gmentation_benchmark_v0_normal]


The following is the same as running:

```bash
python3 test_partseg.py \
     --normal \
     --log_dir pointnet2_part_seg_msg \
     # --log_root ../log \
     # --data_dir ../data/shapenetcore_partanno_segmentation_benchmark_v0_normal
```

In [5]:
args = {
    'normal'  : True, # in source: action='store_true'
    'log_dir' : 'pointnet2_part_seg_msg',
    'log_root': root_log_dir,
    'data_dir': partseg_data_dir,
}
partseg_test_args = test_partseg.CommandLineArgs(**args)
test_metrics, shape_ious, total_correct_class, total_seen_class = test_partseg.main(partseg_test_args, seg_classes)

PARAMETER ...
CommandLineArgs(normal=True, log_dir='pointnet2_part_seg_msg', log_root=PosixPath('/home/gokitune/Documents/pointnet-cleanup/log'), data_dir='/home/gokitune/Documents/pointnet-cleanup/data/shapenetcore_partanno_segmentation_benchmark_v0_normal', batch_size=24, gpu='0', num_point=2048, num_votes=3)
The number of test data is: 2874


100%|██████████| 120/120 [01:54<00:00,  1.04it/s]

eval mIoU of Airplane       0.763466
eval mIoU of Bag            0.450673
eval mIoU of Cap            0.472840
eval mIoU of Car            0.585230
eval mIoU of Chair          0.868757
eval mIoU of Earphone       0.534771
eval mIoU of Guitar         0.877673
eval mIoU of Knife          0.831241
eval mIoU of Lamp           0.757935
eval mIoU of Laptop         0.936523
eval mIoU of Motorbike      0.171415
eval mIoU of Mug            0.748339
eval mIoU of Pistol         0.505387
eval mIoU of Rocket         0.247987
eval mIoU of Skateboard     0.480831
eval mIoU of Table          0.790134
Accuracy is: 0.91024
Class avg accuracy is: 0.63438
Class avg mIOU is: 0.62645
Inctance avg mIOU is: 0.77637


In [6]:
test_metrics, shape_ious

({'accuracy': np.float64(0.9102445959464162),
  'class_avg_accuracy': np.float64(0.6343776818516723),
  'class_avg_iou': np.float64(0.6264501078394253),
  'inctance_avg_iou': np.float64(0.7763717570276613)},
 {'Earphone': np.float64(0.5347707361392766),
  'Motorbike': np.float64(0.17141517727753516),
  'Rocket': np.float64(0.24798672396587237),
  'Car': np.float64(0.5852298315156654),
  'Laptop': np.float64(0.9365225331245296),
  'Cap': np.float64(0.47284048967908004),
  'Skateboard': np.float64(0.48083055195079394),
  'Mug': np.float64(0.7483390471947761),
  'Guitar': np.float64(0.8776732225630821),
  'Bag': np.float64(0.4506727267761233),
  'Lamp': np.float64(0.7579350349604418),
  'Table': np.float64(0.790134138811599),
  'Airplane': np.float64(0.7634661246748955),
  'Pistol': np.float64(0.5053873044878578),
  'Chair': np.float64(0.8687573685480473),
  'Knife': np.float64(0.831240713761229)})

In [7]:
seg_correct = dict(zip(range(len(seg_ids)), total_correct_class))
seg_total = dict(zip(range(len(seg_ids)), total_seen_class))

seg_acc = {}
for id, correct_n in seg_correct.items():
    total_n = seg_total[id]
    if total_n == 0:
        seg_acc[id] = 0
    else:
        seg_acc[id] = correct_n / total_n
# print(seg_acc)

seg_class_acc = {}
for cat in seg_classes:
    seg_class_acc[cat] = {}
    for id in seg_classes[cat]:
        seg_class_acc[cat][id] = seg_acc[id]

pprint(seg_class_acc)

{'Airplane': {0: np.float64(0.9651340528539629),
              1: np.float64(0.8392222059538934),
              2: np.float64(0.7214855552898723),
              3: np.float64(0.6788605815863966)},
 'Bag': {4: np.float64(0.004329004329004329), 5: np.float64(1.0)},
 'Cap': {6: np.float64(1.0), 7: np.float64(0.16944398614090084)},
 'Car': {8: np.float64(0.3254855094114132),
         9: np.float64(0.5420358763777826),
         10: np.float64(0.589101809621892),
         11: np.float64(0.9736415247364153)},
 'Chair': {12: np.float64(0.9629519351777965),
           13: np.float64(0.9321612985488197),
           14: np.float64(0.8910738462789193),
           15: np.float64(0.8667963912966566)},
 'Earphone': {16: np.float64(1.0),
              17: np.float64(0.4034920201882417),
              18: np.float64(0.0)},
 'Guitar': {19: np.float64(0.9629789065863108),
            20: np.float64(0.8530996210070384),
            21: np.float64(0.984170830455918)},
 'Knife': {22: np.float64(0.8545349707

---

## Semantic Segmentation (S3DIS) 


### Data

- S3DIS dataset in text format
- Download `Stanford3dDataset_v1.2.zip` from:
  - [https://cvg-data.inf.ethz.ch/s3dis/](https://cvg-data.inf.ethz.ch/s3dis/)

Default `--data_dir` is `'data/s3dis/Stanford3dDataset_v1.2_Aligned_Version'`.

- `data/`
  - `s3dis/`
    - `ReadMe.txt`
    - `Stanford3dDataset_v1.2_Aligned_Version/`
      - `Area_1/`
        - `conferenceRoom_1/`
          - `Annotations/`
            - `beam_1.txt`
            - ...
          - `conferenceRoom_1.txt`
        - ...
      - ...

### **OUTPUT**

- TRAINING: `<log_root>/semantic_segmentation/<args.log_dir or TIME>/checkpoints/best_model.pth`
- PREDICTION: `None (only prints accuracy)`


### Data preparation

Use `data_utils/collect_indoor3d_data.py` to convert the dataset.

This will create a new directory `data/stanford_indoor3d/` (or whatever you specify with argument).

In [5]:
!python3 data_utils/collect_indoor3d_data.py -h

usage: collect_indoor3d_data.py [-h] [--data_path DATA_PATH]
                                [--output_path OUTPUT_PATH] [--skip_existing]

options:
  -h, --help            show this help message and exit
  --data_path DATA_PATH
                        Path to the indoor3d data directory. Default:
                        data/s3dis/Stanford3dDataset_v1.2_Aligned_Version
  --output_path OUTPUT_PATH
                        Path where processed data will be saved. Default:
                        data/stanford_indoor3d
  --skip_existing       If output file already exist, skip rather than
                        overwriting.


```bash
python3 data_utils/collect_inddoor3d_data.py \
    --data_path data/s3dis/Stanford3dDataset_v1.2_Alligned_Version \
    --output_path data/stanford_indoor3d \
    # --skip_existing
```

In [6]:
collect_indoor3d_data(
    data_path     = semseg_raw_data_dir,
    output_path   = semseg_data_dir,
    skip_existing = False,
)

/home/gokitune/Documents/pointnet-cleanup/data/s3dis/Stanford3dDataset_v1.2_Aligned_Version/Area_1/conferenceRoom_1/Annotations
/home/gokitune/Documents/pointnet-cleanup/data/s3dis/Stanford3dDataset_v1.2_Aligned_Version/Area_1/conferenceRoom_1/Annotations/door_1.txt
/home/gokitune/Documents/pointnet-cleanup/data/s3dis/Stanford3dDataset_v1.2_Aligned_Version/Area_1/conferenceRoom_1/Annotations/chair_10.txt
/home/gokitune/Documents/pointnet-cleanup/data/s3dis/Stanford3dDataset_v1.2_Aligned_Version/Area_1/conferenceRoom_1/Annotations/chair_3.txt
/home/gokitune/Documents/pointnet-cleanup/data/s3dis/Stanford3dDataset_v1.2_Aligned_Version/Area_1/conferenceRoom_1/Annotations/wall_4.txt
/home/gokitune/Documents/pointnet-cleanup/data/s3dis/Stanford3dDataset_v1.2_Aligned_Version/Area_1/conferenceRoom_1/Annotations/wall_3.txt
/home/gokitune/Documents/pointnet-cleanup/data/s3dis/Stanford3dDataset_v1.2_Aligned_Version/Area_1/conferenceRoom_1/Annotations/chair_7.txt
/home/gokitune/Documents/pointnet-

### Training

`train_semseg.py` is used to train the model.

Check all the arguments:

In [10]:
!python3 train_semseg.py -h

usage: Model [-h] [--model MODEL] [--batch_size BATCH_SIZE] [--epoch EPOCH]
             [--learning_rate LEARNING_RATE] [--gpu GPU]
             [--optimizer OPTIMIZER] [--log_dir LOG_DIR]
             [--decay_rate DECAY_RATE] [--npoint NPOINT]
             [--step_size STEP_SIZE] [--lr_decay LR_DECAY]
             [--test_area TEST_AREA] [--data_dir DATA_DIR]

options:
  -h, --help            show this help message and exit
  --model MODEL         model name [default: pointnet_sem_seg]
  --batch_size BATCH_SIZE
                        Batch Size during training [default: 16]
  --epoch EPOCH         Epoch to run [default: 32]
  --learning_rate LEARNING_RATE
                        Initial learning rate [default: 0.001]
  --gpu GPU             GPU to use [default: GPU 0]
  --optimizer OPTIMIZER
                        Adam or SGD [default: Adam]
  --log_dir LOG_DIR     Log path [default: None]
  --decay_rate DECAY_RATE
                        weight decay [default: 1e-4]
  --npoint NP

The following is the same as running:

```bash
python3 train_semseg.py \
     --model pointnet2_sem_seg \
     --test_area 5 \
     --log_dir pointnet2_sem_seg
```

In [7]:
args = {
    'model'    : 'pointnet2_sem_seg',
    'test_area': 5,
    'log_dir'  : 'pointnet2_sem_seg',
    'log_root' : root_log_dir,
    'data_dir' : semseg_data_dir,
}
semseg_train_args = train_semseg.CommandLineArgs(**args)
train_semseg.main(semseg_train_args)

PARAMETER ...
CommandLineArgs(model='pointnet2_sem_seg', test_area=5, log_dir='pointnet2_sem_seg', log_root=PosixPath('/home/gokitune/Documents/pointnet-cleanup/log'), data_dir='/home/gokitune/Documents/pointnet-cleanup/data/stanford_indoor3d', batch_size=16, epoch=32, learning_rate=0.001, gpu='0', optimizer='Adam', decay_rate=0.0001, npoint=4096, step_size=10, lr_decay=0.7)
start loading training data ...


100%|██████████| 204/204 [00:13<00:00, 14.95it/s]


[1.124833  1.1816078 1.        2.2412012 2.340336  2.343587  1.7070498
 2.0335796 1.8852289 3.8252103 1.7948895 2.7857335 1.3452303]
Totally 47623 samples in train set.
start loading test data ...


100%|██████████| 67/67 [00:05<00:00, 12.70it/s]


[1.1381457 1.2059734 1.        9.996554  2.5299199 2.0086675 2.1162353
 1.9657742 2.4815738 4.727607  1.4018297 2.8840992 1.4809785]
Totally 18923 samples in test set.
The number of training data is: 47623
The number of test data is: 18923
No existing model, starting training from scratch...
**** Epoch 1 (1/32) ****
Learning rate:0.001000
BN momentum updated to: 0.100000


...

eval mean loss: 0.962503
eval point avg class IoU: 0.525434
eval point accuracy: 0.818402
eval point avg class acc: 0.610641
------- IoU --------
class ceiling        weight: 0.090, IoU: 0.901 
class floor          weight: 0.200, IoU: 0.976 
class wall           weight: 0.166, IoU: 0.726 
class beam           weight: 0.276, IoU: 0.000 
class column         weight: 0.000, IoU: 0.098 
class window         weight: 0.018, IoU: 0.578 
class door           weight: 0.034, IoU: 0.142 
class table          weight: 0.032, IoU: 0.670 
class chair          weight: 0.039, IoU: 0.757 
class sofa           weight: 0.020, IoU: 0.420 
class bookcase       weight: 0.003, IoU: 0.591 
class board          weight: 0.110, IoU: 0.542 
class clutter        weight: 0.012, IoU: 0.429 

Eval mean loss: 0.962503
Eval accuracy: 0.818402
Best mIoU: 0.530881
**** Epoch 32 (32/32) ****
Learning rate:0.000343
BN momentum updated to: 0.012500



100%|██████████| 2976/2976 [20:07<00:00,  2.46it/s]

Training mean loss: 0.160409
Training accuracy: 0.943613
---- EPOCH 032 EVALUATION ----



100%|██████████| 1182/1182 [07:41<00:00,  2.56it/s]

eval mean loss: 0.944202
eval point avg class IoU: 0.521337
eval point accuracy: 0.809734
eval point avg class acc: 0.633329
------- IoU --------
class ceiling        weight: 0.091, IoU: 0.903 
class floor          weight: 0.200, IoU: 0.961 
class wall           weight: 0.166, IoU: 0.715 
class beam           weight: 0.276, IoU: 0.000 
class column         weight: 0.000, IoU: 0.093 
class window         weight: 0.019, IoU: 0.592 
class door           weight: 0.033, IoU: 0.240 
class table          weight: 0.031, IoU: 0.666 
class chair          weight: 0.040, IoU: 0.747 
class sofa           weight: 0.020, IoU: 0.399 
class bookcase       weight: 0.003, IoU: 0.583 
class board          weight: 0.110, IoU: 0.466 
class clutter        weight: 0.011, IoU: 0.411 

Eval mean loss: 0.944202
Eval accuracy: 0.809734
Best mIoU: 0.530881


### Testing

`test_semseg.py` is used to test the model.

Check all the arguments:

In [ ]:
!python test_semseg.py -h

usage: Model [-h] [--batch_size BATCH_SIZE] [--gpu GPU]
             [--num_point NUM_POINT] --log_dir LOG_DIR [--visual]
             [--test_area TEST_AREA] [--num_votes NUM_VOTES]
             [--data_dir DATA_DIR]

options:
  -h, --help            show this help message and exit
  --batch_size BATCH_SIZE
                        batch size in testing [default: 32]
  --gpu GPU             specify gpu device
  --num_point NUM_POINT
                        point number [default: 4096]
  --log_dir LOG_DIR     experiment root
  --visual              visualize result [default: False]
  --test_area TEST_AREA
                        area for testing, option: 1-6 [default: 5]
  --num_votes NUM_VOTES
                        aggregate segmentation scores with voting [default: 5]
  --data_dir DATA_DIR   data directory [default: data/s3dis/stanford_indoor3d]


The following is the same as running:

```bash
python3 test_semseg.py \
     --log_dir pointnet2_sem_seg \
     --test_area 5 \
     --visual \
     # --log_root ../log \
     # --data_dir ../data/stanford_indoor3d
```

In [8]:
args = {
    'log_dir'  : 'pointnet2_sem_seg',
    'test_area': 5,
    'visual'   : True, # in source: action='store_true'
    'log_root' : root_log_dir,
    'data_dir' : semseg_data_dir,
}
semseg_test_args = test_semseg.CommandLineArgs(**args)
test_semseg.main(semseg_test_args)

PARAMETER ...
CommandLineArgs(log_dir='pointnet2_sem_seg', test_area=5, visual=True, log_root=PosixPath('/home/gokitune/Documents/pointnet-cleanup/log'), data_dir='/home/gokitune/Documents/pointnet-cleanup/data/stanford_indoor3d', batch_size=32, gpu='0', num_point=4096, num_votes=3)
The number of test data is: 67
---- EVALUATION WHOLE SCENE----
Inference [1/67] Area_5_conferenceRoom_2 ...


  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [02:06<00:00, 42.11s/it]


[0.99201421 0.96838355 0.68993227 0.         0.         0.
 0.22175979 0.8892308  0.80147167 0.86681572 0.81127211 0.03081586
 0.5004877 ]
Mean IoU of Area_5_conferenceRoom_2: 0.6157
----------------------------
Inference [2/67] Area_5_office_20 ...


100%|██████████| 3/3 [00:22<00:00,  7.46s/it]


...

[0.96715417 0.99092976 0.62001558 0.         0.         0.
 0.22384043 0.         0.         0.         0.         0.
 0.57485557]
Mean IoU of Area_5_WC_2: 0.6754
----------------------------
Inference [66/67] Area_5_office_23 ...


100%|██████████| 3/3 [00:45<00:00, 15.19s/it]


[0.96798752 0.96918115 0.6355859  0.         0.02031581 0.60508274
 0.01926379 0.81730454 0.86281852 0.         0.85330867 0.87021065
 0.45439905]
Mean IoU of Area_5_office_23: 0.6432
----------------------------
Inference [67/67] Area_5_office_19 ...


100%|██████████| 3/3 [01:48<00:00, 36.22s/it]


[0.94511094 0.98913644 0.72872176 0.         0.         0.
 0.08321192 0.82155857 0.92214491 0.         0.5567404  0.63066012
 0.45949827]
Mean IoU of Area_5_office_19: 0.6819
----------------------------
------- IoU --------
class ceiling       , IoU: 0.904 
class floor         , IoU: 0.980 
class wall          , IoU: 0.733 
class beam          , IoU: 0.000 
class column        , IoU: 0.097 
class window        , IoU: 0.552 
class door          , IoU: 0.200 
class table         , IoU: 0.717 
class chair         , IoU: 0.803 
class sofa          , IoU: 0.471 
class bookcase      , IoU: 0.618 
class board         , IoU: 0.566 
class clutter       , IoU: 0.420 

eval point avg class IoU: 0.543129
eval whole scene point avg class acc: 0.624565
eval whole scene point accuracy: 0.823481
Done!
